In [5]:
import os, sys, re
import sqlite3, csv
import argparse
from tqdm.notebook import tqdm
import json
# %pip install ipywidgets

In [6]:
args = {
    "source_folders": ["/home/gjungwirth/data/02_data/03_analyses/gitleaks_output",
        "/home/gjungwirth/data/02_data/analyses_part2/gitleaks_output"],
    "db": "/home/gjungwirth/data/02_data/results.db"
}

In [49]:
db = sqlite3.connect(args['db'])

db.execute('''
CREATE TABLE IF NOT EXISTS secret (
    secret_id INTEGER PRIMARY KEY,
    file_id INTEGER NOT NULL,
    line TEXT NOT NULL,
    rule TEXT NOT NULL,
    commit_id TEXT NOT NULL,
    author TEXT NOT NULL,
    author_email TEXT NOT NULL,
    commit_date TEXT NOT NULL,
    tags TEXT NOT NULL,
    FOREIGN KEY (file_id) REFERENCES file (file_id),
    UNIQUE (file_id, line, rule, commit_id)
);
''')
db.commit()

In [20]:
src_files = []
for folder in args['source_folders']:
    for root, dirs, filenames in os.walk(folder):
        for filename in filenames:
            repo_folder_name = filename.removesuffix('.json')
            src_files.append([repo_folder_name, os.path.join(root,filename)])

#src_files

In [34]:
# only a test to find historic files

num_total = 0
num_empty = 0

for repo_folder, output_file in src_files[10:30]:
    with open(output_file) as f:
        data = json.load(f)
        for leak in data:
            # print(leak['file'],repo_folder)
            cur = db.execute('''
                SELECT f.file_id
                FROM file f JOIN repo r ON f.repo_id=r.repo_id
                WHERE
                f.file_path = ? AND
                r.folder_name = ?
                ''', 
                (leak['file'], repo_folder)
                )
            if not cur.fetchall():
                num_empty+=1
            num_total+=1

print("Empty: {}/{} = {}%".format(num_empty, num_total, int(num_empty*100/num_total)))

Empty: 0/111 = 0%


In [50]:
pbar = tqdm(total=len(src_files))
res = []
for repo_folder, output_file in src_files:
    with open(output_file) as f:
        data = json.load(f)
        for leak in data:
            # assert( leak['repo'] == repo_folder )
            file_name = re.search(r'([^/]*)$', leak['file']).group(1)
            line_cropped = leak['line'][:200] + (leak['line'][200:] and '...')
            db.execute('''
                INSERT OR IGNORE INTO file
                (repo_id, file_name, file_path, is_deleted, size, sha, mime, no_lines)
                VALUES
                (
                    (SELECT repo_id FROM repo WHERE folder_name = ?),
                    ?, ?,
                    TRUE,
                    -1, "", "", -1
                );
                ''',
                (leak['repo'], file_name, leak['file'])
                )
            db.commit()
            db.execute('''
                INSERT OR IGNORE INTO secret
                (file_id, line, rule, commit_id, author, author_email, commit_date, tags)
                VALUES
                (
                    (SELECT file_id FROM file f JOIN repo r ON f.repo_id=r.repo_id WHERE r.folder_name = ? AND f.file_path = ?),
                    ?,
                    ?,
                    ?,
                    ?,
                    ?,
                    ?,
                    ?
                );''',
                (
                    leak['repo'], leak['file'],
                    line_cropped,
                    leak['rule'],
                    leak['commit'],
                    leak['author'],
                    leak['email'],
                    leak['date'],
                    leak['tags']
                ))
            db.commit()
    pbar.update()



In [51]:
db.close()